### ANOVA is used to test the hypothesis at a significance level of 0.05 against the p-value.
#### If the p-value is lessthan the alpha level of significance we will reject the null hypothesis otherwise we dont reject.
#### To choose between the non parametric and parametric ANOVA , the distribution of the amount column is investigated. If it is normal we use the parametric one way ANOVA otherwise we use the the non prametric ANOVA known as Kruskal Wallis

### Hypothesis 1: <br>
```Null```: The funding received by start-ups in India has not demonstrated consistent upward trajectory over the years.<br>
```Alternate```: The funding received by start-ups in India has demonstrated a consistent upward trajectory over the years.

### Hypothesis 2: <br>
```Null```: There are no significant disparities in funding received by all sectors of the Indian startups. <br>
```Alternate```: The technology sectors receive higher funding compared to other industries.

### Hypothesis 3: <br>
```Null```: Situating a startup in a particular city does not influence funding.<br>
```Alternate```: Situating a startup in a particular city significantly affects funding.

### Hypothesis 4: <br>
```Null```: There are no significant disparities in funding received among different stages of the Indian startups. <br>
```Alternate```: During different stages the startups received different funding amounts.



In [16]:
%pip install spicy
%pip install scikit-posthocs
%pip install pingouin



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/198.6 kB ? eta -:--:--
     --                                       10.2/198.6 kB ? eta -:--:--
     -----                                 30.7/198.6 kB 435.7 kB/s eta 0:00:01
     -----                                 30.7/198.6 kB 435.7 kB/s eta 0:00:01
     -----                                 30.7/198.6 kB 435.7 kB/s eta 0:00:01
     -----                                 30.7/198.6 kB 435.7 kB/s eta 0:00:01
     -----------                           61.4/198.6 kB 192.5 kB/s eta 0:00:01
     -----------------                     92.2/198.6 kB 261.7 kB/s eta 0:00:01
     --------------------                 112.6/198.6 kB 312.2 kB/s eta 0:00:01
     --------------------                 112.6/198.6 kB 312.2 kB/s eta 0:00:01
     ----------------------               122.9/198.6 kB 248.7 kB/s eta 0:00:01
     -------------------------------      174.1/198.6 kB 327.7 kB/s eta 0:00:01
     ------------------------------------ 198.6/198.6


[notice] A new release of pip is available: 23.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import scikit_posthocs as sp
import scipy.stats as stats
from scikit_posthocs import posthoc_dunn
import numpy as np
from scikit_posthocs import posthoc_dunn


In [3]:
## setting the alpha 
alpha=0.05
#reading the data
complete_data=pd.read_excel('complete_dataset.xlsx')
complete_data.sample(5)

,Company Name,Sector,Stage,Amount,HeadQuarter,Year Funded,Investor,Founded,global_sector
1115,Daalchini,Internet of Things,Pre-series A,669000,Undisclosed,2020,Artha Venture Fund,2017.0,Commerce and Shopping
1312,Nykaa,Ecommerce,Undisclosed,8800000,Mumbai,2020,Steadview capital,2012.0,Other
2162,Aqgromalin,AgriTech,Series B,700000,Chennai,2021,Zephyr Peacock,2018.0,Agriculture and Farming
1715,WESS,Renewable Energy,Series B,2000000,Mumbai,2021,Centrum Financial Services,1989.0,Energy
533,Appnomic,Software,Unknown,22000000,Bangalore,2019,Avataar Ventures,2015.0,Science and Engineering


### Using shapiro wilk test to explore distribution of Amounts in each year.

In [4]:
# Group the data by 'Year Funded'
grouped_data = complete_data.groupby('Year Funded')['Amount']

# Perform Shapiro-Wilk test for each group
for year, group in grouped_data:
    statistic, p_value = stats.shapiro(group)
    print(f"Year: {year}")
    print("Shapiro-Wilk Test Results:")
    print("Statistic:", statistic)
    print("P-value:", p_value)
    if p_value < 0.05:
        print("The data does not follow a normal distribution.")
    else:
        print("The data follows a normal distribution.")
    print("-" * 30)

Year: 2018
Shapiro-Wilk Test Results:
Statistic: 0.16452401876449585
P-value: 6.291830104818429e-43
The data does not follow a normal distribution.
------------------------------
Year: 2019
Shapiro-Wilk Test Results:
Statistic: 0.7388180494308472
P-value: 1.5155211807726943e-10
The data does not follow a normal distribution.
------------------------------
Year: 2020
Shapiro-Wilk Test Results:
Statistic: 0.015582144260406494
P-value: 0.0
The data does not follow a normal distribution.
------------------------------
Year: 2021
Shapiro-Wilk Test Results:
Statistic: 0.012447237968444824
P-value: 0.0
The data does not follow a normal distribution.
------------------------------


## The distribution is not normal hence non parametric ANOVA is used

## <font color=green>Hypothesis 1, we will analyze the year-by-year funding amounts and calculate the average growth rate of funding.</font>


In [33]:
## print each amount in each stage as a list and get the amount per stage as an array

amounts_per_year=complete_data.groupby("Year Funded")['Amount'].apply(list)
amount_year = np.array(amounts_per_year)
amount_year


Year Funded
2018    [250000, 5840000, 9490000, 2000000, 3530000, 1...
2019    [6300000, 28000000, 30000000, 6000000, 6000000...
2020    [200000, 100000, 3000000, 400000, 340000, 6000...
2021    [1200000, 120000000, 30000000, 51000000, 20000...
Name: Amount, dtype: object

In [21]:
# Perform one-way ANOVA
test_statistic, p_value = stats.kruskal(*amount_year)
print(f"P_value: {p_value}")


# Interpret the results
if p_value < alpha:
    print("Reject the null hypothesis concluding that there are significant disparities in funding received during different years.")
else:
    print("Fail to reject the null hypothesis concluding there is no significant evidence of disparities in funding among the years.")

P_value: 3.72985152276845e-05
Reject the null hypothesis concluding that there are significant disparities in funding received during different years.


### <font color=green>Conclusion:  There is are significant disparities in funding received during different years.<font>

In [44]:
# Extract "Year Funded" as an array and convert it to strings
year_funded_array = amounts_per_year.index.astype(str).to_numpy()

# Perform Dunn's test with Bonferroni correction
dunn_result = posthoc_dunn(amount_year, p_adjust='bonferroni')

# Modify the Dunn's test result to include custom group names
dunn_result.index = year_funded_array
dunn_result.columns = year_funded_array

print(dunn_result)

          2018      2019      2020      2021
2018  1.000000  0.251144  0.005956  1.000000
2019  0.251144  1.000000  0.001814  0.081846
2020  0.005956  0.001814  1.000000  0.008579
2021  1.000000  0.081846  0.008579  1.000000


### Print out what are significant differences among the years

In [48]:

def group_comparisons_by_pvalues(posthoc_result, alpha=0.05):
    
    is_significant = []  # p < alpha
    not_significant = []  # p > or = alpha
    
    for group1 in posthoc_result.index:
        for group2 in posthoc_result.columns:
            p_value = posthoc_result.loc[group1, group2]
            if p_value < alpha :
                is_significant.append((group1, group2))
            else:
                not_significant.append((group1, group2))
    
    return is_significant, not_significant

# Group the comparisons based on p-values
is_significant, not_significant = group_comparisons_by_pvalues(dunn_result, alpha=0.05)

# Print the results
print("Significant Differences:")
print(is_significant)


print("\nNot Significant Differences:")
print(not_significant)


Significant Differences:
[('2018', '2020'), ('2019', '2020'), ('2020', '2018'), ('2020', '2019'), ('2020', '2021'), ('2021', '2020')]

Not Significant Differences:
[('2018', '2018'), ('2018', '2019'), ('2018', '2021'), ('2019', '2018'), ('2019', '2019'), ('2019', '2021'), ('2020', '2020'), ('2021', '2018'), ('2021', '2019'), ('2021', '2021')]


### The significant differences in funding are between years ('2018', '2020'), ('2019', '2020'), ('2020', '2018'), ('2020', '2019'), ('2020', '2021'), ('2021', '2020')

## <font color=green>Hypothesis 2: Categorize start-ups based on industry and compare the funding amounts received by each sector.</font>


In [6]:
## print each amount in each sector as a list and get the amount per sector as an array

amounts_per_sector=complete_data.groupby("global_sector")['Amount'].apply(list)
amount_sector = np.array(amounts_per_sector)
#amount_sector

In [7]:
# Perform one-way ANOVA
test_statistic, p_value = stats.kruskal(*amount_sector)
print(f"P_value: {p_value}")


# Interpret the results
if p_value < alpha:
    print("Reject the null hypothesis concluding that there are significant disparities in funding received in different sectors.")
else:
    print("Fail to reject the null hypothesis concluding there is no significant evidence of disparities in funding among the sectors.")

P_value: 6.7883496534318e-11
Reject the null hypothesis concluding that there are significant disparities in funding received in different sectors.


### <font color=green> Conclusion : There are significant disparities in funding received in different sectors.</font>

### <font color=green>Investigating diffference between technology and other sectors funding</font>

In [9]:

# Empty lists to store funding amounts for the "Tech" sector and other sectors
tech_sector_amounts = []
other_sector_amounts = []

# Loop through the DataFrame and group funding amounts based on sector
for index, row in complete_data.iterrows():
    if 'Artificial Intelligence' or 'tech' or 'Data' in row['Sector']:
        tech_sector_amounts.append(row['Amount'])
    else:
        other_sector_amounts.append(row['Amount'])

# Perform Mann-Whitney U test
test_statistic, p_value = stats.kruskal(tech_sector_amounts, other_sector_amounts)

# Set the significance level
alpha = 0.05

# Interpret the results
if p_value < alpha:
    print("Reject the null hypothesis. There is a significant difference in funding amounts between the Tech sector and other sectors.")
else:
    print("Fail to reject the null hypothesis. There is no significant evidence of a difference in funding amounts between the Tech sector and other sectors.")


Fail to reject the null hypothesis. There is no significant evidence of a difference in funding amounts between the Tech sector and other sectors.


### <font color=green>Hypothesis 3, Examine the distribution of start-ups across cities and deduce which cities harbour most highly-funded start-ups.</font>


In [ ]:
## print each amount in each city as a list and get the amount per city as an array

amounts_per_city=complete_data.groupby("HeadQuarter")['Amount'].apply(list)
amount_city = np.array(amounts_per_city)
amount_city

In [59]:
# Perform one-way ANOVA
test_statistic, p_value = stats.kruskal(*amount_city)
print(f"P_value: {p_value}")


# Interpret the results
if p_value < alpha:
    print("Reject the null hypothesis concluding that there are significant disparities in funding received in cities.")
else:
    print("Fail to reject the null hypothesis concluding there is no significant evidence of disparities in funding among the cities.")

P_value: 5.361887072236649e-07
Reject the null hypothesis concluding that there are significant disparities in funding received in cities.


### <font color=green>Conclusion : There are significant disparities in funding received in cities.</font>

### Investigate the differences

In [68]:
# Extract "Year Funded" as an array and convert it to strings
city_funded_array = amounts_per_city.index.astype(str).to_numpy()

# Perform Dunn's test with Bonferroni correction
dunn_result = posthoc_dunn(amount_city, p_adjust='bonferroni')

# Modify the Dunn's test result to include custom group names
dunn_result.index = city_funded_array
dunn_result.columns = city_funded_array

#dunn_result

In [69]:

def group_comparisons_by_pvalues(posthoc_result, alpha=0.05):
    
    is_significant = []  # p < alpha
    not_significant = []  # p > or = alpha
    
    for group1 in posthoc_result.index:
        for group2 in posthoc_result.columns:
            p_value = posthoc_result.loc[group1, group2]
            if p_value < alpha :
                is_significant.append((group1, group2))
            else:
                not_significant.append((group1, group2))
    
    return is_significant, not_significant

# Group the comparisons based on p-values
is_significant, not_significant = group_comparisons_by_pvalues(dunn_result, alpha=0.05)

# Print the results
print("Significant Differences:")
print(is_significant)


print("\nNot Significant Differences:")
print(not_significant)


Significant Differences:
[('Bangalore', 'Hyderabad'), ('Hyderabad', 'Bangalore'), ('Hyderabad', 'Outside India'), ('Outside India', 'Hyderabad')]

Not Significant Differences:
[('Ahmadabad', 'Ahmadabad'), ('Ahmadabad', 'Ahmedabad'), ('Ahmadabad', 'Alleppey'), ('Ahmadabad', 'Alwar'), ('Ahmadabad', 'Ambernath'), ('Ahmadabad', 'Anand'), ('Ahmadabad', 'Andheri'), ('Ahmadabad', 'Azadpur'), ('Ahmadabad', 'Bangalore'), ('Ahmadabad', 'Bangalore City'), ('Ahmadabad', 'Banglore'), ('Ahmadabad', 'Beijing'), ('Ahmadabad', 'Belgaum'), ('Ahmadabad', 'Bengaluru'), ('Ahmadabad', 'Bhilwara'), ('Ahmadabad', 'Bhopal'), ('Ahmadabad', 'Bhubaneswar'), ('Ahmadabad', 'Bihar'), ('Ahmadabad', 'Chandigarh'), ('Ahmadabad', 'Chennai'), ('Ahmadabad', 'Cochin'), ('Ahmadabad', 'Coimbatore'), ('Ahmadabad', 'Computer Games'), ('Ahmadabad', 'Dehradun'), ('Ahmadabad', 'Delhi'), ('Ahmadabad', 'Dhingsara'), ('Ahmadabad', 'Ernakulam'), ('Ahmadabad', 'Faridabad'), ('Ahmadabad', 'Faridabad, Haryana'), ('Ahmadabad', 'Gandhinag

### Conclusion: There are significant differences among these cities ('Bangalore', 'Hyderabad'), ('Hyderabad', 'Bangalore'), ('Hyderabad', 'Outside India'), ('Outside India','Hyderabad')

## <font color=green>Hypothesis 4: Categorize start-ups based on stages and compare the funding amounts received by each stage.</font>

In [ ]:
## print each amount in each stage as a list and get the amount per stage as an array

amounts_per_stage=complete_data.groupby("Stage")['Amount'].apply(list)
amount_stage = np.array(amounts_per_stage)
amount_stage

In [64]:
# Perform one-way ANOVA
test_statistic, p_value = stats.kruskal(*amount_stage)
print(f"P_value: {p_value}")


# Interpret the results
if p_value < alpha:
    print("Reject the null hypothesis concluding that there are significant disparities in funding received during different stages.")
else:
    print("Fail to reject the null hypothesis concluding there is no significant evidence of disparities in funding among the stages.")

P_value: 1.9523873835796214e-120
Reject the null hypothesis concluding that there are significant disparities in funding received during different stages.


### <font color=green>Conclusion: There are significant disparities in funding received during different stages</font>

### Find where the differences are

In [70]:
# Extract "Year Funded" as an array and convert it to strings
stage_funded_array = amounts_per_stage.index.astype(str).to_numpy()

# Perform Dunn's test with Bonferroni correction
dunn_result = posthoc_dunn(amount_stage, p_adjust='bonferroni')

# Modify the Dunn's test result to include custom group names
dunn_result.index = stage_funded_array
dunn_result.columns = stage_funded_array

#dunn_result

In [71]:

def group_comparisons_by_pvalues(posthoc_result, alpha):
    
    is_significant = []  # p < alpha
    not_significant = []  # p > or = alpha
    
    for group1 in posthoc_result.index:
        for group2 in posthoc_result.columns:
            p_value = posthoc_result.loc[group1, group2]
            if p_value < alpha :
                is_significant.append((group1, group2))
            else:
                not_significant.append((group1, group2))
    
    return is_significant, not_significant

# Group the comparisons based on p-values
is_significant, not_significant = group_comparisons_by_pvalues(dunn_result, 0.05)

# Print the results
print("Significant Differences:")
print(is_significant)


print("\nNot Significant Differences:")
print(not_significant)


Significant Differences:
[('Angel', 'Private Equity'), ('Angel', 'Series C'), ('Angel', 'Series D'), ('Angel', 'Series E'), ('Bridge', 'Private Equity'), ('Bridge', 'Series C'), ('Bridge', 'Series D'), ('Bridge', 'Series E'), ('Debt', 'Pre-seed'), ('Debt', 'Series C'), ('Debt Financing', 'Pre-seed'), ('Debt Financing', 'Pre-series'), ('Debt Financing', 'Pre-series A'), ('Debt Financing', 'Seed'), ('Debt Financing', 'Series C'), ('Post-IPO Equity', 'Pre-seed'), ('Pre-Seed', 'Private Equity'), ('Pre-Seed', 'Series C'), ('Pre-Seed', 'Series D'), ('Pre-Seed', 'Series E'), ('Pre-seed', 'Debt'), ('Pre-seed', 'Debt Financing'), ('Pre-seed', 'Post-IPO Equity'), ('Pre-seed', 'Private Equity'), ('Pre-seed', 'Secondary Market'), ('Pre-seed', 'Series A'), ('Pre-seed', 'Series B'), ('Pre-seed', 'Series C'), ('Pre-seed', 'Series D'), ('Pre-seed', 'Series E'), ('Pre-seed', 'Undisclosed'), ('Pre-seed', 'Unknown'), ('Pre-seed', 'Venture - Series Unknown'), ('Pre-series', 'Debt Financing'), ('Pre-series